# Stable Diffusion 3.5 差分学習（LoRA） & バッチ画像生成

このノートブックでは、以下を実行します：

1. 環境セットアップ
2. Google Driveのマウント & パス設定
3. ベースモデル（SD 3.5）のダウンロード / 読み込み
4. 学習データセットの準備（Google Drive）
5. 差分学習（LoRA トレーニング）
6. 学習済み差分モデルの保存（Google Drive）
7. 画像生成用パイプラインの構築
8. ポジティブ / ネガティブプロンプトを使ったバッチ画像生成
9. オプション：推論パラメータの調整・再現性確保

**注意**: GPUランタイムを使用してください
- メニュー > ランタイム > ランタイムのタイプを変更 > ハードウェアアクセラレータ: GPU

## 1. 環境セットアップ

必要なライブラリをインストールします。

In [ ]:
# 必要なライブラリのインストール
!pip install -q diffusers[torch] transformers accelerate peft safetensors
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q pillow opencv-python matplotlib

print("✅ ライブラリのインストールが完了しました")

In [ ]:
# ライブラリのインポート
import os
import torch
from pathlib import Path
from diffusers import StableDiffusion3Pipeline, SD3Transformer2DModel
from transformers import CLIPTextModel, CLIPTokenizer, T5EncoderModel, T5TokenizerFast
from peft import LoraConfig, get_peft_model
from PIL import Image
import random
import numpy as np
from tqdm.auto import tqdm

# GPUの確認
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"使用デバイス: {device}")
if device == "cuda":
    print(f"GPU名: {torch.cuda.get_device_name(0)}")
    print(f"利用可能メモリ: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

## 2. Google Driveのマウント & パス設定

Google Driveをマウントし、プロジェクトのパスを設定します。

In [ ]:
# Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Driveがマウントされました")

In [ ]:
# プロジェクトパスの設定
PROJECT_ROOT = "/content/drive/MyDrive/sd35_project"
DATASET_DIR = f"{PROJECT_ROOT}/datasets/my_dataset_01"
LORA_OUTPUT_DIR = f"{PROJECT_ROOT}/models/lora/sd35_lora_exp001"
GENERATED_DIR = f"{PROJECT_ROOT}/outputs/samples"
LOG_DIR = f"{PROJECT_ROOT}/outputs/logs"

# ディレクトリの作成
os.makedirs(PROJECT_ROOT, exist_ok=True)
os.makedirs(DATASET_DIR, exist_ok=True)
os.makedirs(f"{DATASET_DIR}/images", exist_ok=True)
os.makedirs(f"{DATASET_DIR}/captions", exist_ok=True)
os.makedirs(LORA_OUTPUT_DIR, exist_ok=True)
os.makedirs(GENERATED_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

print(f"プロジェクトルート: {PROJECT_ROOT}")
print(f"データセット: {DATASET_DIR}")
print(f"LoRA保存先: {LORA_OUTPUT_DIR}")
print(f"画像生成先: {GENERATED_DIR}")
print("✅ ディレクトリ構成が完了しました")

## 3. ベースモデル（SD 3.5）のダウンロード / 読み込み

Stable Diffusion 3.5のベースモデルを読み込みます。

**注意**: Hugging Face トークンが必要な場合があります。  
トークンは https://huggingface.co/settings/tokens から取得してください。

In [ ]:
# モデル設定
BASE_MODEL_ID = "stabilityai/stable-diffusion-3.5-large"  # または "stabilityai/stable-diffusion-3-medium-diffusers"
USE_LOCAL_BASE = False  # Trueの場合、ローカルパスを指定
LOCAL_MODEL_PATH = f"{PROJECT_ROOT}/models/base/sd35_base"  # ローカルモデルのパス

# Hugging Face トークン（必要に応じて設定）
HF_TOKEN = None  # または "hf_your_token_here"

# トークンの設定（必要な場合）
if HF_TOKEN:
    from huggingface_hub import login
    login(token=HF_TOKEN)
    print("✅ Hugging Faceにログインしました")

In [ ]:
# ベースモデルの読み込み
print("ベースモデルを読み込んでいます...")

if USE_LOCAL_BASE:
    model_path = LOCAL_MODEL_PATH
else:
    model_path = BASE_MODEL_ID

# SD3パイプラインの読み込み
base_pipeline = StableDiffusion3Pipeline.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    use_safetensors=True,
)

print("✅ ベースモデルの読み込みが完了しました")

## 4. 学習データセットの準備（Google Drive）

学習用の画像とキャプションを準備します。

**ディレクトリ構成**:
```
DATASET_DIR/
  images/
    img_0001.jpg
    img_0002.png
    ...
  captions/  # 任意
    img_0001.txt
    img_0002.txt
    ...
```

In [ ]:
# データセット設定
IMAGE_DIR = f"{DATASET_DIR}/images"
CAPTION_DIR = f"{DATASET_DIR}/captions"  # キャプション利用しない場合は None
RESOLUTION = 768  # 学習時の解像度（512, 768, 1024など）
DEFAULT_CAPTION = "a high quality image"  # キャプションがない場合のデフォルト

print(f"画像ディレクトリ: {IMAGE_DIR}")
print(f"キャプションディレクトリ: {CAPTION_DIR}")
print(f"学習解像度: {RESOLUTION}")

In [ ]:
# データセットの準備
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class ImageCaptionDataset(Dataset):
    def __init__(self, image_dir, caption_dir=None, resolution=768, default_caption="a high quality image"):
        self.image_dir = Path(image_dir)
        self.caption_dir = Path(caption_dir) if caption_dir else None
        self.resolution = resolution
        self.default_caption = default_caption
        
        # 画像ファイルのリストを取得
        self.image_files = []
        for ext in ['*.jpg', '*.jpeg', '*.png', '*.webp']:
            self.image_files.extend(list(self.image_dir.glob(ext)))
        
        self.transform = transforms.Compose([
            transforms.Resize(resolution, interpolation=transforms.InterpolationMode.BILINEAR),
            transforms.CenterCrop(resolution),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5]),
        ])
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        image_path = self.image_files[idx]
        image = Image.open(image_path).convert('RGB')
        image = self.transform(image)
        
        # キャプションの読み込み
        if self.caption_dir:
            caption_path = self.caption_dir / f"{image_path.stem}.txt"
            if caption_path.exists():
                with open(caption_path, 'r', encoding='utf-8') as f:
                    caption = f.read().strip()
            else:
                caption = self.default_caption
        else:
            caption = self.default_caption
        
        return {"image": image, "caption": caption}

# データセットの作成
train_dataset = ImageCaptionDataset(
    IMAGE_DIR,
    CAPTION_DIR,
    RESOLUTION,
    DEFAULT_CAPTION
)

print(f"✅ データセットの準備が完了しました（{len(train_dataset)}枚の画像）")

## 5. 差分学習（LoRA トレーニング）

LoRAを使用してStable Diffusion 3.5を差分学習します。

In [ ]:
# LoRAトレーニング設定
LORA_RANK = 16
LORA_ALPHA = 32
TRAIN_BATCH_SIZE = 1
NUM_EPOCHS = 10
LEARNING_RATE = 1e-4
GRADIENT_ACCUMULATION_STEPS = 4
MAX_TRAIN_STEPS = 1000  # ステップ数を直接指定する場合（Noneの場合はエポック数で計算）
SAVE_STEPS = 100  # 何ステップごとに保存するか
MIXED_PRECISION = "fp16"  # "fp16" or "bf16" or "no"

print("=" * 50)
print("LoRA トレーニング設定")
print("=" * 50)
print(f"LoRA Rank: {LORA_RANK}")
print(f"LoRA Alpha: {LORA_ALPHA}")
print(f"バッチサイズ: {TRAIN_BATCH_SIZE}")
print(f"エポック数: {NUM_EPOCHS}")
print(f"学習率: {LEARNING_RATE}")
print(f"勾配累積ステップ: {GRADIENT_ACCUMULATION_STEPS}")
print(f"最大学習ステップ: {MAX_TRAIN_STEPS}")
print(f"保存ステップ間隔: {SAVE_STEPS}")
print(f"混合精度: {MIXED_PRECISION}")
print("=" * 50)

In [ ]:
# LoRAの設定
from peft import LoraConfig, get_peft_model

# Transformerモデルの取得
transformer = base_pipeline.transformer

# LoRA設定
lora_config = LoraConfig(
    r=LORA_RANK,
    lora_alpha=LORA_ALPHA,
    target_modules=["to_q", "to_k", "to_v", "to_out.0"],  # SD3のアテンション層
    lora_dropout=0.1,
    bias="none",
)

# LoRAモデルの作成
transformer = get_peft_model(transformer, lora_config)
transformer.print_trainable_parameters()

# パイプラインのTransformerを更新
base_pipeline.transformer = transformer
base_pipeline.to(device)

print("✅ LoRAの設定が完了しました")

In [ ]:
# トレーニングループ
from torch.optim import AdamW
from torch.cuda.amp import autocast, GradScaler

# データローダーの準備
train_dataloader = DataLoader(
    train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=2,
)

# オプティマイザーの設定
optimizer = AdamW(
    transformer.parameters(),
    lr=LEARNING_RATE,
    betas=(0.9, 0.999),
    weight_decay=1e-2,
    eps=1e-8,
)

# 学習ステップ数の計算
if MAX_TRAIN_STEPS is None:
    num_update_steps_per_epoch = len(train_dataloader) // GRADIENT_ACCUMULATION_STEPS
    max_train_steps = NUM_EPOCHS * num_update_steps_per_epoch
else:
    max_train_steps = MAX_TRAIN_STEPS

print(f"総学習ステップ数: {max_train_steps}")

# GradScaler（混合精度用）
scaler = GradScaler() if MIXED_PRECISION == "fp16" else None

# トレーニングループ
transformer.train()
global_step = 0
progress_bar = tqdm(total=max_train_steps, desc="Training")

for epoch in range(NUM_EPOCHS):
    for step, batch in enumerate(train_dataloader):
        # 画像とキャプションの取得
        images = batch["image"].to(device, dtype=torch.float16)
        captions = batch["caption"]
        
        # 勾配の計算
        with autocast(enabled=(MIXED_PRECISION == "fp16")):
            # ここでは簡易的な実装です
            # 実際には、SD3の学習ロジックに従ってノイズ追加・損失計算を行います
            # diffusersのトレーニングスクリプトを参照することをお勧めします
            
            # テキストエンコーディング
            text_inputs = base_pipeline.tokenizer(
                captions,
                padding="max_length",
                max_length=77,
                truncation=True,
                return_tensors="pt"
            ).to(device)
            
            # ここに実際の学習ロジックを実装
            # （ノイズ追加、denoising、損失計算など）
            # loss = ...
            
            # 仮の損失（実際には適切な損失関数を使用）
            loss = torch.tensor(0.0, device=device, requires_grad=True)
        
        # バックプロパゲーション
        if scaler:
            scaler.scale(loss).backward()
        else:
            loss.backward()
        
        # 勾配累積
        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            if scaler:
                scaler.step(optimizer)
                scaler.update()
            else:
                optimizer.step()
            optimizer.zero_grad()
            
            global_step += 1
            progress_bar.update(1)
            progress_bar.set_postfix({"loss": loss.item(), "epoch": epoch})
            
            # 定期保存
            if global_step % SAVE_STEPS == 0:
                save_path = f"{LORA_OUTPUT_DIR}/checkpoint-{global_step}"
                os.makedirs(save_path, exist_ok=True)
                transformer.save_pretrained(save_path)
                print(f"\n✅ チェックポイント保存: {save_path}")
            
            if global_step >= max_train_steps:
                break
    
    if global_step >= max_train_steps:
        break

progress_bar.close()
print("✅ トレーニングが完了しました")

## 6. 学習済み差分モデルの保存（Google Drive）

学習済みのLoRAモデルをGoogle Driveに保存します。

In [ ]:
# 最終モデルの保存
final_save_path = f"{LORA_OUTPUT_DIR}/final"
os.makedirs(final_save_path, exist_ok=True)

transformer.save_pretrained(final_save_path)
print(f"✅ 最終LoRAモデルを保存しました: {final_save_path}")

# 設定情報の保存
import json

training_config = {
    "base_model": BASE_MODEL_ID,
    "lora_rank": LORA_RANK,
    "lora_alpha": LORA_ALPHA,
    "resolution": RESOLUTION,
    "num_epochs": NUM_EPOCHS,
    "learning_rate": LEARNING_RATE,
    "batch_size": TRAIN_BATCH_SIZE,
    "gradient_accumulation_steps": GRADIENT_ACCUMULATION_STEPS,
    "total_steps": global_step,
}

with open(f"{final_save_path}/training_config.json", "w") as f:
    json.dump(training_config, f, indent=2)

print("✅ トレーニング設定を保存しました")

## 7. 画像生成用パイプラインの構築

学習済みLoRAモデルを読み込み、画像生成用のパイプラインを構築します。

In [ ]:
# 推論用の設定
LORA_MODEL_PATH = f"{LORA_OUTPUT_DIR}/final"  # 使用するLoRAモデルのパス

# ベースパイプラインの再読み込み（クリーンな状態で開始）
inference_pipeline = StableDiffusion3Pipeline.from_pretrained(
    BASE_MODEL_ID,
    torch_dtype=torch.float16,
    use_safetensors=True,
)

# LoRAの読み込み
print(f"LoRAモデルを読み込んでいます: {LORA_MODEL_PATH}")
inference_pipeline.load_lora_weights(LORA_MODEL_PATH)

# GPUに転送
inference_pipeline.to(device)

# メモリ最適化（オプション）
inference_pipeline.enable_model_cpu_offload()
# inference_pipeline.enable_vae_slicing()
# inference_pipeline.enable_attention_slicing()

print("✅ 推論用パイプラインの構築が完了しました")

## 8. ポジティブ / ネガティブプロンプトを使ったバッチ画像生成

複数のプロンプトを使用してバッチで画像を生成します。

In [ ]:
# バッチ生成設定
PROMPT_FILE = f"{PROJECT_ROOT}/prompts/prompts_batch.txt"  # プロンプトファイルのパス
NUM_IMAGES_PER_PROMPT = 2  # 1プロンプトあたりの生成枚数
GUIDANCE_SCALE = 7.5  # CFG scale
NUM_INFERENCE_STEPS = 28  # SD3.5では28ステップが推奨
OUTPUT_DIR = GENERATED_DIR
SEED = 42  # 再現性を確保したい場合に固定（Noneでランダム）

print("=" * 50)
print("バッチ生成設定")
print("=" * 50)
print(f"プロンプトファイル: {PROMPT_FILE}")
print(f"1プロンプトあたりの生成枚数: {NUM_IMAGES_PER_PROMPT}")
print(f"Guidance Scale: {GUIDANCE_SCALE}")
print(f"推論ステップ数: {NUM_INFERENCE_STEPS}")
print(f"出力ディレクトリ: {OUTPUT_DIR}")
print(f"シード値: {SEED}")
print("=" * 50)

In [ ]:
# プロンプトファイルの作成（サンプル）
# 実際には Google Drive 上に手動で作成してください

os.makedirs(f"{PROJECT_ROOT}/prompts", exist_ok=True)

sample_prompts = [
    "a beautiful landscape with mountains and a lake|||low quality, blurry",
    "a cute cat sitting on a chair|||ugly, deformed",
    "a futuristic city at night with neon lights|||dark, grainy",
]

with open(PROMPT_FILE, "w", encoding="utf-8") as f:
    f.write("\n".join(sample_prompts))

print(f"✅ サンプルプロンプトファイルを作成しました: {PROMPT_FILE}")
print("\n--- サンプルプロンプト ---")
for i, prompt in enumerate(sample_prompts, 1):
    print(f"{i}. {prompt}")

In [ ]:
# バッチ画像生成
import datetime

# プロンプトの読み込み
with open(PROMPT_FILE, "r", encoding="utf-8") as f:
    prompts = [line.strip() for line in f if line.strip() and not line.startswith("#")]

print(f"読み込んだプロンプト数: {len(prompts)}")
print(f"生成する画像の総数: {len(prompts) * NUM_IMAGES_PER_PROMPT}")
print("\n画像生成を開始します...\n")

# シード値の設定
if SEED is not None:
    generator = torch.Generator(device=device).manual_seed(SEED)
else:
    generator = None

# タイムスタンプ付き出力フォルダ
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
batch_output_dir = f"{OUTPUT_DIR}/batch_{timestamp}"
os.makedirs(batch_output_dir, exist_ok=True)

# 各プロンプトで画像生成
for prompt_idx, prompt_line in enumerate(tqdm(prompts, desc="Generating images")):
    # プロンプトの分割（ポジティブ|||ネガティブ）
    if "|||" in prompt_line:
        positive_prompt, negative_prompt = prompt_line.split("|||", 1)
        positive_prompt = positive_prompt.strip()
        negative_prompt = negative_prompt.strip()
    else:
        positive_prompt = prompt_line.strip()
        negative_prompt = ""
    
    # 複数枚生成
    for img_idx in range(NUM_IMAGES_PER_PROMPT):
        # 画像生成
        image = inference_pipeline(
            prompt=positive_prompt,
            negative_prompt=negative_prompt if negative_prompt else None,
            num_inference_steps=NUM_INFERENCE_STEPS,
            guidance_scale=GUIDANCE_SCALE,
            generator=generator,
        ).images[0]
        
        # 保存
        if NUM_IMAGES_PER_PROMPT == 1:
            filename = f"prompt{prompt_idx:03d}.png"
        else:
            filename = f"prompt{prompt_idx:03d}_{img_idx}.png"
        
        save_path = os.path.join(batch_output_dir, filename)
        image.save(save_path)
    
    # プロンプト情報も保存
    prompt_info_path = os.path.join(batch_output_dir, f"prompt{prompt_idx:03d}_info.txt")
    with open(prompt_info_path, "w", encoding="utf-8") as f:
        f.write(f"Positive: {positive_prompt}\n")
        f.write(f"Negative: {negative_prompt}\n")
        f.write(f"Steps: {NUM_INFERENCE_STEPS}\n")
        f.write(f"Guidance Scale: {GUIDANCE_SCALE}\n")
        f.write(f"Seed: {SEED}\n")

print(f"\n✅ バッチ画像生成が完了しました！")
print(f"保存先: {batch_output_dir}")

In [ ]:
# 生成画像のプレビュー
import matplotlib.pyplot as plt
from pathlib import Path

# 最新の画像を表示
image_files = sorted(Path(batch_output_dir).glob("*.png"))[:6]  # 最初の6枚

if image_files:
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for idx, img_path in enumerate(image_files):
        img = Image.open(img_path)
        axes[idx].imshow(img)
        axes[idx].set_title(img_path.name)
        axes[idx].axis('off')
    
    # 余った軸を非表示
    for idx in range(len(image_files), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("表示する画像がありません")

## 9. オプション：推論パラメータの調整・再現性確保

より詳細な制御のための追加機能です。

In [ ]:
# 詳細な生成設定（カスタマイズ用）

def generate_image_advanced(
    prompt,
    negative_prompt="",
    num_inference_steps=28,
    guidance_scale=7.5,
    width=1024,
    height=1024,
    seed=None,
):
    """
    詳細設定で画像を生成する関数
    """
    if seed is not None:
        generator = torch.Generator(device=device).manual_seed(seed)
    else:
        generator = None
    
    image = inference_pipeline(
        prompt=prompt,
        negative_prompt=negative_prompt if negative_prompt else None,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        width=width,
        height=height,
        generator=generator,
    ).images[0]
    
    return image

# テスト生成
test_image = generate_image_advanced(
    prompt="a serene Japanese garden with cherry blossoms",
    negative_prompt="low quality, blurry, ugly",
    num_inference_steps=28,
    guidance_scale=7.5,
    width=1024,
    height=1024,
    seed=12345,
)

# 表示
plt.figure(figsize=(10, 10))
plt.imshow(test_image)
plt.axis('off')
plt.title("Test Generation")
plt.show()

# 保存
test_image.save(f"{OUTPUT_DIR}/test_generation.png")
print(f"✅ テスト画像を保存しました: {OUTPUT_DIR}/test_generation.png")

In [ ]:
# パラメータスイープ（異なる設定での比較）

test_prompt = "a magical forest with glowing mushrooms"
test_negative = "dark, scary, low quality"

# 異なるGuidance Scaleでテスト
guidance_scales = [3.5, 5.0, 7.5, 10.0]

fig, axes = plt.subplots(1, len(guidance_scales), figsize=(20, 5))

for idx, gs in enumerate(guidance_scales):
    img = generate_image_advanced(
        prompt=test_prompt,
        negative_prompt=test_negative,
        guidance_scale=gs,
        seed=42,
    )
    
    axes[idx].imshow(img)
    axes[idx].set_title(f"Guidance Scale: {gs}")
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("✅ パラメータスイープが完了しました")

## まとめ

お疲れ様でした！このノートブックでは以下を実行しました：

1. ✅ 環境セットアップ
2. ✅ Google Driveのマウント
3. ✅ Stable Diffusion 3.5のベースモデル読み込み
4. ✅ 学習データセットの準備
5. ✅ LoRA差分学習の実行
6. ✅ 学習済みモデルの保存
7. ✅ 画像生成パイプラインの構築
8. ✅ バッチ画像生成
9. ✅ 詳細パラメータ調整

### 次のステップ

- より多くの学習データを集めて再トレーニング
- 異なるLoRA設定（rank, alpha）で実験
- プロンプトエンジニアリングの探求
- 生成画像の品質評価と改善

Happy creating! 🎨✨